# Importanto Bibliotecas

In [17]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50

# Caminho das pastas
AUTHORIZED_DIR = 'authorized_faces'
EMBEDDING_DIR = 'embedding'

# Carrega o modelo base (MobileNetV2)

In [18]:
base_model = ResNet50(input_shape=(224, 224, 3),
                      include_top=False,
                      pooling='avg',
                      weights='imagenet')

model = Model(inputs=base_model.input, outputs=base_model.output)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 37s 0us/step


# Função para gerar embedding facial

In [19]:
def get_embedding(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Erro ao ler imagem: {image_path}")
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img.astype(np.float32))
    img = np.expand_dims(img, axis=0)
    embedding = model.predict(img, verbose=0)
    return embedding.flatten()

# Função para calcular similaridade (cosine similarity)

In [20]:
def cosine_similarity(a, b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return np.dot(a, b)

# Gera e salva os embeddings

In [21]:
def generate_and_save_embeddings(authorized_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for user in os.listdir(authorized_dir):
        user_path = os.path.join(authorized_dir, user)
        if os.path.isdir(user_path):
            embeddings = []
            for img_name in os.listdir(user_path):
                img_path = os.path.join(user_path, img_name)
                try:
                    emb = get_embedding(img_path)
                    embeddings.append(emb)
                except Exception as e:
                    print(f"[Erro] {e}")
            if embeddings:
                np.save(os.path.join(output_dir, f"{user}.npy"), np.array(embeddings))
                print(f"✅ Embeddings salvos: {user}")

# Executa
generate_and_save_embeddings(AUTHORIZED_DIR, EMBEDDING_DIR)

✅ Embeddings salvos: Jonathan Breck
✅ Embeddings salvos: Josh Radnor
✅ Embeddings salvos: Konstantin Khabenskiy
✅ Embeddings salvos: Kristen Wiig
✅ Embeddings salvos: Sharon Leal
✅ Embeddings salvos: Jenn Proske
✅ Embeddings salvos: Bella Thorne
✅ Embeddings salvos: Zach Galifianakis
✅ Embeddings salvos: Franka Potente
✅ Embeddings salvos: Sarah Jessica Parker
✅ Embeddings salvos: Zack Ward
✅ Embeddings salvos: Tiffany Haddish
✅ Embeddings salvos: Jack Black
✅ Embeddings salvos: Alice Englert
✅ Embeddings salvos: Jonathan D Mellor
✅ Embeddings salvos: Jeremy Sumpter
✅ Embeddings salvos: Marion Cotillard
✅ Embeddings salvos: Diane Lane
✅ Embeddings salvos: Emily Mortimer
✅ Embeddings salvos: Joan Allen
✅ Embeddings salvos: Lubna Azabal
✅ Embeddings salvos: Tahar Rahim
✅ Embeddings salvos: Harrison Ford
✅ Embeddings salvos: Jason Drucker
✅ Embeddings salvos: Bel Powley
✅ Embeddings salvos: Alicia Vikander
✅ Embeddings salvos: Kang-ho Song
✅ Embeddings salvos: Chris Pratt
✅ Embeddings sal

# Load Savad Embeddings

In [22]:
def load_saved_embeddings(embedding_dir):
    user_embeddings = {}
    for file in os.listdir(embedding_dir):
        if file.endswith('.npy'):
            name = file.replace('.npy', '')
            embeddings = np.load(os.path.join(embedding_dir, file))
            user_embeddings[name] = embeddings
    return user_embeddings

# Banco de embeddings autorizados

In [23]:
# Carrega todos os embeddings dos usuários autorizados
def load_authorized_embeddings(authorized_dir):
    user_embeddings = {}
    for user_folder in os.listdir(authorized_dir):
        user_path = os.path.join(authorized_dir, user_folder)
        if os.path.isdir(user_path):
            embeddings = []
            for img_file in os.listdir(user_path):
                img_path = os.path.join(user_path, img_file)
                try:
                    emb = get_embedding(img_path)
                    embeddings.append(emb)
                except Exception as e:
                    print(f"[Erro] {e}")
            if embeddings:
                user_embeddings[user_folder] = embeddings
    return user_embeddings    

# Verificar acesso comparando com múltiplas imagens por usuário

In [46]:
def verify_access(input_image_path, user_embeddings, threshold=0.7):
    input_embedding = get_embedding(input_image_path)
    for user, embeddings_list in user_embeddings.items():
        for ref_embedding in embeddings_list:
            similarity = cosine_similarity(input_embedding, ref_embedding)
            # print(f"Comparando com {user} - Similaridade: {similarity:.2f}")
            if similarity > threshold:
                print(f"✅ Acesso Liberado para {user}")
                return True
    print("❌ Acesso Negado")
    return False

# Teste de verificação

In [47]:
authorized_embeddings = load_saved_embeddings(EMBEDDING_DIR)

In [48]:
authorized_embeddings.keys()

dict_keys(['Angelina Jolie', 'Logan Lerman', 'Linda Hamilton', 'Ioan Gruffudd', 'Jay Hernandez', 'Russell Brand', 'Anna Paquin', 'Julia Louis-Dreyfus', 'Joel Edgerton', 'Brett Dalton', 'Matt Passmore', 'Daniella Alonso', 'Megan Fox', 'Kristen Bell', 'Josephine Langford', 'Adelaide Clemens', 'Natasha Calis', 'Elsie Fisher', 'Rosa Salazar', 'Paul Giamatti', 'Josh Brolin', 'Jimmy Bennett', 'Kate Bosworth', 'Kate Beckinsale', 'Rupert Everett', 'Toshio Furukawa', 'Mel Gibson', 'Octavia Spencer', 'Regina Hall', 'Shane Harper', 'Rhys Ifans', 'Viola Davis', 'Andy Samberg', 'Jonathan Breck', 'Scarlett Johansson', 'Emma Stone', 'Amanda Bynes', 'Jason Momoa', 'Carlos Alazraqui', 'Brooklynn Prince', 'Stephen Campbell Moore', 'Josh Zuckerman', 'John Cameron Mitchell', 'Nathan Fillion', 'Woody Allen', 'Claire Foy', 'Elisha Cuthbert', 'Michael Fassbender', 'Sarah Butler', 'Julie Andrews', 'Douglas Smith', 'Nat Wolff', 'John Cusack', 'Ashley Tisdale', 'Katie Featherston', 'Noomi Rapace', 'David Arquet

In [49]:
authorized_embeddings['Angelina Jolie'].shape

(5, 2048)

In [50]:
TEST_IMAGE = 'test_images/Adam Sandler.jpg'
# Executa verificação
status = verify_access(TEST_IMAGE, authorized_embeddings, threshold=0.9)

✅ Acesso Liberado para Adam Sandler


In [51]:
TEST_IMAGE = 'test_images/Octavia Spencer.jpg'
# Executa verificação
status = verify_access(TEST_IMAGE, authorized_embeddings, threshold=0.9)

✅ Acesso Liberado para Octavia Spencer


In [52]:
TEST_IMAGE = 'test_images/James Franco.jpg'
# Executa verificação
status = verify_access(TEST_IMAGE, authorized_embeddings, threshold=0.9)

✅ Acesso Liberado para Robert Pattinson
